## Pbc4cip- Python Implementation

In [1]:
import sys
import weka.core.jvm as jvm
import weka.core.packages as packages
from weka.core.classes import complete_classname
from weka.core.converters import Loader
from weka.classifiers import Classifier
import weka.plot.graph as graph  # NB: pygraphviz and PIL are required
from weka.core.classes import Random, from_commandline
import weka.core.serialization as serialization
from weka.filters import Filter
import re
from tqdm import tqdm
import time
import pandas as pd
import pickle

In [2]:
#Michael
#wekafiles_path = "C:/Users/mzenk/wekafiles"
#PBC4CIP_zip_path = "C:/Users/mzenk/Google Drive/ITESM/Maestría/Semestre 3/ML2/Assignment4/PBC4cip-1.0-weka.zip"
#data_dir = "C:/Users/mzenk/Google Drive/ITESM/Maestría/Semestre 3/ML2/Assignment4/Exped_Visualizations/"
#arff_file = "weka_dataset_clean_noop_nocorreo.arff"

In [3]:
#Dany
wekafiles_path = "/Users/dannygc/wekafiles"
PBC4CIP_zip_path = "/Users/dannygc/Google Drive File Stream/My Drive/MCCNotes/MCC 3/Adv_ML/PBC_HW4/PBC4cip-1.0-weka.zip"
data_dir = "/Volumes/GoogleDrive/My Drive/MCCNotes/Jlab projects/GITHUB_repositories/DRAE_repositories/Exped_Visualizations/"
arff_file = "weka_dataset_clean_.arff"

In [4]:
#Define Constraints
updated_PBC4CIP = False
min_support_diff = 0.3
min_count_diff = 30
min_ratio = 0.6
max_patterns = 15

In [5]:
#Parameters
trees = 150
maxDepth = 5
min_objectsByLeaf = 35

In [6]:

attname = 'Forma del procedimiento'
attnum = 0
filename = attname + str(attnum)

# Dictionary with the nominal attributes in order, and their respective onehot indexes
# Only change this if an attribute is moved or deleted
""" nom_pos = {
    'Caracter del procedimiento' : [18,19],
    'Forma del procedimiento' : [7,8,9],
    'Operador' : [],
    'Correo electronico' : [],
    'Entidad federativa' : [10,11,12,13,14,15],
    'Tipo de contratación' : [21,22,23,24,25],
    'Articulo' : [18,19,20],
    'Plantilla' : [26,27,28,29,30,31]
} """
nom_pos = {
    'Caracter del procedimiento' : [18,19],
    'Forma del procedimiento' : [9,10,11],
    'Operador' : [],
    'Correo electronico' : [],
    'Entidad federativa' : [12,13,14,15,16,17],
    'Tipo de contratación' : [23,24,25,26,27],
    'Articulo' : [20,22,22],
    'Plantilla' : [28,29,30,31,32,33]
}
class_index = len(nom_pos.keys()) - 1

In [7]:
len(nom_pos.keys())

8

In [8]:
nom_pos

{&#39;Caracter del procedimiento&#39;: [18, 19],
 &#39;Forma del procedimiento&#39;: [9, 10, 11],
 &#39;Operador&#39;: [],
 &#39;Correo electronico&#39;: [],
 &#39;Entidad federativa&#39;: [12, 13, 14, 15, 16, 17],
 &#39;Tipo de contratación&#39;: [23, 24, 25, 26, 27],
 &#39;Articulo&#39;: [20, 22, 22],
 &#39;Plantilla&#39;: [28, 29, 30, 31, 32, 33]}

In [9]:
pickle_file = 'dany.pickle'
load_pickle = True
save_pickle = True

In [10]:
if save_pickle:
    with open(pickle_file, 'wb') as f:
        pickle.dump([   wekafiles_path,
                        PBC4CIP_zip_path,
                        data_dir,
                        arff_file,
                        updated_PBC4CIP], f)

In [11]:
if load_pickle:
    with open(pickle_file, 'rb') as f:
        wekafiles_path, PBC4CIP_zip_path, data_dir, arff_file, updated_PBC4CIP = pickle.load(f)

In [12]:
start_time = time.time()
#jvm.start(packages=True, max_heap_size="12g") #max_heap_size 512m, 4g. packages=true searches for weka packages in installation program
jvm.start(packages=wekafiles_path, max_heap_size='12g')

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=[&#39;/Users/dannygc/anaconda3/lib/python3.7/site-packages/javabridge/jars/rhino-1.7R4.jar&#39;, &#39;/Users/dannygc/anaconda3/lib/python3.7/site-packages/javabridge/jars/runnablequeue.jar&#39;, &#39;/Users/dannygc/anaconda3/lib/python3.7/site-packages/javabridge/jars/cpython.jar&#39;, &#39;/Users/dannygc/anaconda3/lib/python3.7/site-packages/weka/lib/python-weka-wrapper.jar&#39;, &#39;/Users/dannygc/anaconda3/lib/python3.7/site-packages/weka/lib/weka.jar&#39;]
DEBUG:weka.core.jvm:MaxHeapSize=12g
DEBUG:weka.core.jvm:Using alternative Weka home directory: /Users/dannygc/wekafiles
DEBUG:weka.core.jvm:Using alternative Weka home directory: /Users/dannygc/wekafiles


In [13]:
pkg = "PBC4cip"
print(complete_classname("." + pkg))
# install package if necessary 
if not packages.is_installed(pkg):
    print("Installing %s..." % pkg)
    #packages.install_package("http://prdownloads.sourceforge.net/weka/discriminantAnalysis1.0.3.zip?download")
    packages.install_package(PBC4CIP_zip_path)
    print("Installed %s, please re-run script!" % pkg)
    jvm.stop()
    sys.exit(0)
else:
    print(pkg + " is already installed.")

weka.classifiers.trees.PBC4cip
PBC4cip is already installed.


In [14]:
# testing classname completion

print("\n\n\n\n\n")
print(">>> Start...")

loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file(data_dir + arff_file)

cmdline = []







&gt;&gt;&gt; Start...


# Preprocess
Here the one-hot attributes not used are deleted.
Also the nominal attribute where the class being evaluated comes from is also deleted.

In [15]:
attributes = [f.name for f in data.attributes()]
attributes

[&#39;Caracter del procedimiento&#39;,
 &#39;Forma del procedimiento&#39;,
 &#39;Operador&#39;,
 &#39;Correo electronico&#39;,
 &#39;Entidad federativa&#39;,
 &#39;Tipo de contratacion&#39;,
 &#39;Articulo&#39;,
 &#39;Plantilla&#39;,
 &#39;Forma_Procedimiento_Presencial&#39;,
 &#39;Forma_Procedimiento_Electronica&#39;,
 &#39;Forma_Procedimiento_Mixta&#39;,
 &#39;Entidad_federativa_Norte&#39;,
 &#39;Entidad_federativa_Centro&#39;,
 &#39;Entidad_federativa_Golfo&#39;,
 &#39;Entidad_federativa_PacificoSur&#39;,
 &#39;Entidad_federativa_Sur&#39;,
 &#39;Entidad_federativa_PacificoNorte&#39;,
 &#39;Caracter del procedimiento_Nacional&#39;,
 &#39;Caracter del procedimiento_Internacional&#39;,
 &#39;Articulo de excepcion_41&#39;,
 &#39;Articulo de excepcion_42&#39;,
 &#39;Articulo de excepcion_43&#39;,
 &#39;Contratacion_adquisiciones&#39;,
 &#39;Contratacion_servicios&#39;,
 &#39;Contratacion_obra_publica&#39;,
 &#39;Contratacion_arrendamientos&#39;,
 &#39;Contratacion_servicios_op&#39;,
 &#3

In [17]:
pos_vector = [False for i in range(len(attributes))]
pos_vector[0:len(nom_pos.keys())] = [True for i in range(len(nom_pos.keys()))]
pos_vector[len(attributes)-9:len(attributes)] = [True for i in range(9)]

In [18]:
pos_vector

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [19]:

# Dict that will contain filtered datasets
filtered_data = {
    'Caracter del procedimiento' : [],
    'Forma del procedimiento' : [],
    'Entidad federativa' : [],
    'Tipo de contratación' : [],
    'Articulo' : [],
    'Plantilla' : []
}
for key, value in nom_pos.items():
    for onehot_att in value:
        # Here modify pos vector then feed it into weka remove
        # Change nom att to false
        # Change only specific onehot to true
        # See how to save it to change the class also
        nominal_att_index = list(nom_pos.keys()).index(key)
        pos_vector[nominal_att_index] = False
        pos_vector[onehot_att-1] = True
        indeces_not_filtered = [i+1 for i, val in enumerate(pos_vector) if val]
        pos_vector[nominal_att_index] = True
        pos_vector[onehot_att-1] = False
        remove = Filter(classname="weka.filters.unsupervised.attribute.Remove", options=["-R",",".join(map(str, indeces_not_filtered)),"-V"])
        remove.inputformat(data)
        filtered_data[key].append(remove.filter(data))
            

# Criterions

In [29]:
data = filtered_data[attname][attnum]
data.class_index = class_index

print("Class attribute is: " + [f.name for f in data.attributes()][class_index])
print(f'Max Depth is: {maxDepth}')
print(f'Number of Trees is: {trees}')
print(f'Min Objects by Leaf is: {min_objectsByLeaf}')

input_config = f'weka.classifiers.trees.PBC4cip -S 1 -miner \"PRFramework.Core.SupervisedClassifiers.EmergingPatterns.Miners.RandomForestMinerWithFiltering -bagSizePercent 100 -numFeatures -1 -numTrees {trees} -builder \\\"PRFramework.Core.SupervisedClassifiers.DecisionTrees.Builder.DecisionTreeBuilder -distributionEvaluator \\\\\\\"PRFramework.Core.SupervisedClassifiers.DecisionTrees.DistributionEvaluators.Hellinger \\\\\\\" -maxDepth {maxDepth} \\\\\\\"-minimalObjByLeaf \\\\\\\" {min_objectsByLeaf} -minimalSplitGain 1.0E-30\\\"\"'


#weka.classifiers.trees.PBC4cip -S 1 -miner "PRFramework.Core.SupervisedClassifiers.EmergingPatterns.Miners.RandomForestMinerWithoutFiltering -bagSizePercent 100 -numFeatures -1 -numTrees 150 -builder \"PRFramework.Core.SupervisedClassifiers.DecisionTrees.Builder.DecisionTreeBuilder -distributionEvaluator \\\"PRFramework.Core.SupervisedClassifiers.DecisionTrees.DistributionEvaluators.Hellinger \\\" -maxDepth 5 \\\"-minimalObjByLeaf \\\" 35 -minimalSplitGain 1.0E-30\""


classifier = from_commandline(input_config, classname="weka.classifiers.Classifier")

print(">>> Building classifier...")
start_time_1 = time.time()
classifier.build_classifier(data)
print(f">>> [Done] Bulding classifier. Time: {(time.time() - start_time_1)} seconds ---")

print(">>> Serializing model...")
start_time_1 = time.time()
classifier.serialize(f"{filename}.model")
print(f">>> [Done] Serializing model. Time: {(time.time() - start_time_1)} seconds ---")

print(">>> Generating big string...")
start_time_1 = time.time()
big_string = str(classifier).split("]\n")
print(f">>> [Done] Generating big string. Time: {(time.time() - start_time_1)} seconds ---")

In [21]:
c0_count = 0
c1_count = 0

for instance in data:
    if instance.values[class_index] == 0:
        c0_count += 1
    else:
        c1_count += 1
print('c0_count: ', c0_count)
print('c1_count: ', c1_count)
print('Total: ', c0_count + c1_count)

c0_count:  48999
c1_count:  47123
Total:  96122


In [22]:
list_fields = []
list_support_c0 = []
list_support_c1 = []
list_num_c0 = []
list_num_c1 = []
list_diff = []
with tqdm(total=len(big_string)) as pbar:
    for item in big_string:
        text = ""
        text = item + "]"
        text = text.split("[")
        fields = text[0]
        # print(fields)
        if updated_PBC4CIP:
            if (len(text) > 1):
                class_nums = text[1]
                class_nums = class_nums[:-1]
                class_nums = class_nums.split()
                supports = text[2]
                supports = supports[:-1]
                supports = supports.split()
                list_fields.append(fields.strip())
                list_support_c0.append(float(supports[0]))
                list_support_c1.append(float(supports[1]))
                list_num_c0.append(float(class_nums[0]))
                list_num_c1.append(float(class_nums[1]))
        else:
            if (len(text) > 1):
                supports = text[1]
                supports = supports[:-1]
                supports = supports.split()
                list_fields.append(fields.strip())
                list_support_c0.append(float(supports[0]))
                list_support_c1.append(float(supports[1]))
                list_num_c0.append(float(supports[0])*c0_count)
                list_num_c1.append(float(supports[1])*c1_count)
        pbar.update(1)

100%|██████████| 2094/2094 [00:00&lt;00:00, 75226.74it/s]


In [26]:
df = pd.DataFrame(columns = ['patterns', 'support_c0', 'support_c1', 'num_c0', 'num_c1'])
df['patterns']=list_fields
df['support_c0']=list_support_c0
df['support_c1']= list_support_c1
df['num_c0']= list_num_c0
df['num_c1']= list_num_c1
df.tail()

,patterns,support_c0,support_c1,num_c0,num_c1
2088,Operador != 'irma olga arredondo murillo' AND ...,0.72,0.60,35279.28,28273.80
2089,Caracter del procedimiento = 'nacional' AND Pl...,0.54,0.80,26459.46,37698.40
2090,Caracter del procedimiento = 'nacional' AND Mo...,0.41,0.81,20089.59,38169.63
2091,Plantilla != 'proyecto' AND Operador != 'irma ...,0.83,0.49,40669.17,23090.27
2092,Caracter del procedimiento = 'nacional' AND Mo...,0.56,0.89,27439.44,41939.47


In [27]:
df['s_diff'] = df.support_c1 - df.support_c0
df['n_diff'] = df.num_c1 - df.num_c0
df['ratio'] = df.num_c1/(df.num_c0 + df.num_c1)

df = df[df.s_diff > min_support_diff]
df = df[df.n_diff > min_count_diff]
df = df[df.ratio > min_ratio]

df = df.sort_values(by=['s_diff'], ascending=False)
df = df[:max_patterns]

df.head()

,patterns,support_c0,support_c1,num_c0,num_c1,s_diff,n_diff,ratio
2081,Most_used_UC_word_gyr = '0' AND Caracter del p...,0.18,0.68,8819.82,32043.64,0.50,23223.82,0.784164
2080,Caracter del procedimiento = 'nacional' AND Ar...,0.18,0.68,8819.82,32043.64,0.50,23223.82,0.784164
2075,Articulo != '42' AND Caracter del procedimient...,0.18,0.66,8819.82,31101.18,0.48,22281.36,0.779068
2066,Most_used_UC_word_gyr = '0' AND Plantilla = 'a...,0.16,0.64,7839.84,30158.72,0.48,22318.88,0.793681
2065,Most_used_UC_word_gyr = '0' AND Plantilla = 'a...,0.16,0.64,7839.84,30158.72,0.48,22318.88,0.793681


## Export to Patterns Obtained

In [25]:
df.to_csv(f"Patterns_Obtained/{filename}.csv", index=False)
jvm.stop()